**Elaboração de um novo modelo de classificação com base nas informações de usuários avaliados pelo INCT-DD**

In [ ]:
#Carrega as bibliotecas
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, explained_variance_score, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import math
import statistics
import datetime
import pytz
import pickle
## NLTK (biblioteca para processamento de linguagem natural)
import nltk
from nltk.stem.rslp import RSLPStemmer ##http://www.nltk.org/howto/portuguese_en.html

#O primeiro uso exige obter os pacotes adicionais da biblioteca descomentando as linhas a seguir
#Instala os pacotes de termos do nltk (apenas na primeira vez)
#nltk.download()
#nltk.download('rslp')

**O novo modelo de classificação de bots foi construído com base nos usuários manualmente avaliados pelo INCT-DD**

Essa escolha foi tomada considerando que esse conjunto de dados é o melhor que se possui quanto à real possibilidade de um usuário do Twitter ser um bot, não existindo bases de avaliação dentro da realidade brasileira (especialmente quanto ao português), bem como atualizadas

In [ ]:
#Busca os dados dos usuários avaliados
datafile_users = "data/sample2/inct_users.csv"
df_users = pd.read_csv(datafile_users, header = 0)

#Preenche os valores NaN con 0 apenas para avaliação geral
df_users = df_users.fillna(0)
print(len(df_users))
#Apresenta o total de usuários avaliados
df_users.head()

**No novo modelos são consideradas apenas as informações associadas como "É bot?" de respotas "Sim" ou "Não"**

In [ ]:
#Busca a classificação do INCT-DD
datafile_handles = "data/sample1/handles_inct.csv" #A classificação é a mesma da sample1
df_handles = pd.read_csv(datafile_handles, header = 0)
print(len(df_handles))
df_handles['É Bot?'].head()

**As mais recentes postagens dos usuários foram consideradas como um atributo do modelo**

Para a classificação dos usuários, o novo modelo inclui atributos relacionados com as postagens dos usuários, na tentativa de extrair informação mais atualizada e dinâmica de sua atuação. Entretanto, os textos das postagens foram utilizados unificando seus conteúdos e extraindo informações representativas, tais como os termos mais recorrentemente utilizados, diferença no tempo das postagens e repostagens

In [ ]:
#Recupera os últimos twittes
datafile_timeline = "data/sample2/inct_timelines.csv"
df_timeline = pd.read_csv(datafile_timeline, header = 0)
print(len(df_timeline))
df_timeline.head()

Aplica um pré-processamento nos dados para unificar a informação da postagens se tratar de um retweet

In [ ]:
#identifica os formatos existentes
df_timeline['tweet_is_retweet'].unique()

In [ ]:
df_timeline['retweet_tratado'] = df_timeline['tweet_is_retweet'].apply(lambda x: "sim" if (x == 'True' or x == True) else "não")
df_timeline['retweet_tratado'].unique()

In [ ]:
#Necessário reverificar no texto do tweet por RT @, pois o campo tweet_is_retweet falha em algumas situações não identificadas
#Parecem ser os RT com comentários adicionais
#for tweet in df_timeline['retweet_tratado', 'tweet_text']:
#    if tweet['retweet_tratado'] == 'não':
#        if tweet['tweet_text'].find("RT @") != -1:
#            tweet['retweet_tratado'] = 'sim'
#len(df_timeline)
#for i in range(len(df_timeline)):
#    if df_timeline.iloc[i]['retweet_tratado'] == 'não':
#        if df_timeline.iloc[i]['tweet_text'].find("RT @") != -1:
#            df_timeline.iloc[i]['retweet_tratado']  = 'sim'
df_timeline['tweet_com_rt_tratado'] = df_timeline['tweet_text'].apply(lambda x: "sim" if x.find("RT @") != -1 else "não" )

In [ ]:
#Combina em uma única coluna as informações de retweets e tweets com RT comentados
def reune_rt(retweet,rt):
    if retweet == 'sim' or rt == 'sim':
        return 'sim'
    else:
        return 'não'

df_timeline['retweet_e_tweet_com_rt_tratado'] = df_timeline.apply(lambda x: reune_rt(x.retweet_tratado, x.tweet_com_rt_tratado), axis=1)

In [ ]:
df_timeline[df_timeline["retweet_e_tweet_com_rt_tratado"] == 'sim']

Extrai a diferença em segundos entre as postagens do usuário

In [ ]:
#Incluir uma dedida da distancia temporal entre twittes (mediana e mínimo)
df_handles['Tempo mediano'] = np.array(len(df_handles))
df_handles['Tempo menor']   = np.array(len(df_handles))
iuser = 0
for user in df_handles['handle']:
    df_temp = df_timeline[df_timeline['tweet_author'] == user]
    itweet = 0
    menor = 100000
    difs = list()
    tweet_date_prev = None
    for tweet in df_temp['tweet_created_at']:
        tweet_date = pd.to_datetime(pd.to_datetime(tweet).strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
        if itweet > 0:
            dif = (tweet_date_prev - tweet_date).seconds
            if dif < menor:
                menor = dif
            difs.append(dif)
        else:
            tweet_date_prev = tweet_date
        tweet_date_prev = tweet_date
        itweet += 1
    if len(difs) > 0:
        mediana = statistics.median(difs)
    else:
        mediana = 1000
    print(user + ' - ' + str(menor) + ' - ' + str(mediana)+'\n')
    df_handles['Tempo mediano'][iuser] = mediana
    df_handles['Tempo menor'][iuser]   = menor
    iuser += 1
    
    

**Os dados inicialmente tratados são reunidos com a classificação dada pelo INCT-DD**

In [ ]:
#Reune os dados do usuário com a classificação
df_result_merge = pd.merge(df_handles, df_users, on=['handle'])
print(len(df_result_merge))
df_result_merge.head()

**Os dados das postagens foram reunidos para a extração de informações representativas**

Para viabilizar o treinamento do modelo, os dados por postagens foram convertidos em conjuntos por usuário (autor do tweet, e a representação foi dada por informações sumarizadas ou probabilísticas, por exemplo, as hashtags mais utilizadas ou o percentual de postagens realizadas a partir do Android, iPhone ou Web.

In [ ]:
#Reune todos os tweets de um mesmo autor em um único texto, separando apenas por vírgula
df_result_text = df_timeline.groupby('tweet_author').agg({'tweet_text':lambda col: ', '.join(col)}).reset_index()

In [ ]:
#Reune todos as hashtags utilizadas por um mesmo autor em um único texto, separando apenas por vírgula
df_result_hashtags = df_timeline.groupby('tweet_author').agg({'tweet_hashtags':lambda col: ', '.join(col)}).reset_index()

In [ ]:
#Reune a informação de fonte de todos os tweets de um mesmo autor em um único texto, separando apenas por vírgula
df_result_source = df_timeline.groupby('tweet_author').agg({'tweet_source':lambda col: ', '.join(col)}).reset_index()

In [ ]:
#Reune as informações de twettes que são retweets
df_result_retweet = df_timeline.groupby('tweet_author').agg({'retweet_tratado':lambda col: ', '.join(col)}).reset_index()

In [ ]:
#Reune as informações de twettes com RT
df_result_tweet_com_rt = df_timeline.groupby('tweet_author').agg({'tweet_com_rt_tratado':lambda col: ', '.join(col)}).reset_index()
df_result_tweet_com_rt

In [ ]:
#Reune as informações da junção de retweets e tweets com rt
df_result_retweet_e_tweet_com_rt = df_timeline.groupby('tweet_author').agg({'retweet_e_tweet_com_rt_tratado':lambda col: ', '.join(col)}).reset_index()
df_result_retweet_e_tweet_com_rt

In [ ]:
#Reune os dados (merge) do usuários, suas avaliações com texto dos tweets, as hashtags, as fontes e os retweets
df_result_merge = pd.merge(df_handles, df_users, on=['handle'])
df_result_merge = pd.merge(df_result_merge,df_result_text, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_hashtags, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_source, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_retweet, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_tweet_com_rt, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_retweet_e_tweet_com_rt, left_on=['handle'], right_on=['tweet_author'])

In [ ]:
#Exibe parte dos resultados da junção (nem todos os usuários ainda estão ativos e número de amostras diminui)
print(len(df_result_merge))
df_result_merge.head()

**A classificação dos usuários foi padronizada para 0 - Não Bot e 1 - Bot**

In [ ]:
#Padroniza a saída da classificação do INCT-DD para bot e monta o conjunto Y
df = df_result_merge
y = df['É Bot?'].apply(lambda x: 1 if (x == 'Sim' or x == 'sim') else 0)
y.reset_index(drop=True, inplace=True)
y.head()

In [ ]:
##Seleciona as colunas para o conjunto X
#feature_cols = ['tweet_text'] #,'tweet_source','tweet_hashtags'
#x = df['tweet_text']
#x.shape

** [Classficando apenas pelo texto dos Twittes (NLTK)] **

In [ ]:
##Prepara o conjunto de dados para treinamento e teste
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) 

In [ ]:
##Método para vetorizar e contabilizar os termos
stemmer = nltk.stem.RSLPStemmer()
class StemmedCountVectorizerRSLPS(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizerRSLPS, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizerRSLPS(stop_words=nltk.corpus.stopwords.words('portuguese'))
tfidf_transformer = TfidfTransformer()

In [ ]:
##Pipeline para extrair as informaçoes e classificar com base no texto (pode ser usado ANN ou MNB [MultinomialNB(fit_prior=False)])
#text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
#                      ('tfidf', TfidfTransformer()),
#                      ('mnb', MLPClassifier(random_state=1, max_iter=600, activation='relu',solver='adam')),
#])
#text_mnb_stemmed = text_mnb_stemmed.fit(x_train, y_train)

In [ ]:
#text_mnb_stemmed

In [ ]:
##Avalia a classificação
#predicted_mnb_stemmed = text_mnb_stemmed.predict(x_test)
#np.mean(predicted_mnb_stemmed == y_test)

**Os atributos do treinamentos envolvem diversos fatores**

Uma das etapas mais critícas da modelagem é a definição dos atributos que representam o cenário real, nesse sentido foram incluídas o máximo de variáveis que pudessem representar um usuário e suas atividades na rede, desde o tamanho do login escolhido até o tempo mínimo entre suas postagens. Na sequência são realizadas as atividades de extração, tratamento e junção dessas informações como atributos do conjunto de treinamento do modelo.

In [ ]:
df.columns #df é o conjunto completo de dados, já com os twittes-hashtags-sources-retweets em campos únicos

In [ ]:
df.head()

De todo os conjuntos de informações disponíveis não foram selecionados aquelas que não poderiam ser automaticamente extraídos dos perfis e atividades dos usuários na rede. Portanto, as classificações como "comportamento agressivo?", "Parece só Retweetar?", entre outras, não foram incluídos no conjunto de treinamento.

In [ ]:
feature_cols = ['followers_count', 'friends_count', 'Tempo mediano', 'Tempo menor']
x = df[feature_cols]

In [ ]:
##Converte os testos em frequências
#st = stemmed_count_vect.fit_transform((df['tweet_text']))
#tfidf_transformer = TfidfTransformer()
#x_tfidf = tfidf_transformer.fit_transform(st)
#x_tfidf

In [ ]:
##Inclui as frequências no conjunto x
#x_tfidf.shape
#x.join(pd.DataFrame(x_tfidf.todense()))

In [ ]:
len(df['tweet_hashtags'][7].replace("[","").replace("]","").replace(", \'","$").split("$"))
len(df['tweet_hashtags'][7].split(", ["))

In [ ]:
#Inclui os quantitativos de hashtages utilizadas (e a mediana por postagem)

qtd_hashtags = df['tweet_hashtags'].apply(lambda x: len(x.replace("[","").replace("]","").replace(", \'","$").split("$")))
x['Quantidade hashtags'] = np.array(list(qtd_hashtags))
qtd_hashtags_media = df['tweet_hashtags'].apply(lambda x: len(x.replace("[","").replace("]","").replace(", \'","$").split("$"))/len(x.split(", [")))
x['Quantidade hashtags media'] = np.array(list(qtd_hashtags_media))

x.head()

In [ ]:
#Inclui o número de dígitos no nome
username_digitos = df['handle'].apply(lambda x: sum(c.isdigit() for c in str(x)) ) 
x['Digitos no username'] = np.array(list(username_digitos))

In [ ]:
#O tamanho do nome e do login
tam_username = df['handle'].apply(lambda x: len(str(x)))
tam_nome = df['name'].apply(lambda x: len(str(x)))
x['Tamanho do username'] = np.array(list(tam_username))
x['Tamanho do nome'] = np.array(list(tam_nome))

In [ ]:
x.head()

A fonte do tweet foi considera importante informação, considerando que automações de postagens possam ser facilitadas a partir da versão Web ou que possa existir algum padrão no uso das diferentes fontes. Sendo assim, forneceu-se ao métodos a informação percentual da origem das postagens do mesmo usuário, seja Android, iPhone ou Web.

In [ ]:
#Calcula a quantidade de twittes por fontes
fonte_android = df['tweet_source'].apply(lambda x: str(x).count('Twitter for Android') )
fonte_iphone = df['tweet_source'].apply(lambda x: str(x).count('Twitter for iPhone') )
fonte_web = df['tweet_source'].apply(lambda x: str(x).count('Twitter Web App') )

In [ ]:
fonte_soma = fonte_android + fonte_iphone + fonte_web
fonte_soma = fonte_soma.apply(lambda x: 1 if x <= 0 else x )

In [ ]:
#Calcula o percentual por usuário
fonte_android = fonte_android/fonte_soma
fonte_iphone = fonte_iphone/fonte_soma
fonte_web = fonte_web/fonte_soma

In [ ]:
x['Fonte de Android'] = np.array(list(fonte_android))
x['Fonte de iPhone'] = np.array(list(fonte_iphone))
x['Fonte de Web'] = np.array(list(fonte_web))
x = x.fillna(0)
x.head()

In [ ]:
#Avaliação geral das diferentes fontes
x['Fonte de Android'].describe()

In [ ]:
x['Fonte de iPhone'].describe()

In [ ]:
x['Fonte de Web'].describe()

In [ ]:
#Inclui a informação do retweet
df['retweet_tratado'].head()

In [ ]:
retweet_tratado = df['retweet_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['retweet_tratado_media'] = np.array(list(retweet_tratado))

In [ ]:
tweet_com_rt = df['tweet_com_rt_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['tweet_com_rt_tratado_media'] = np.array(list(tweet_com_rt))

In [ ]:
retweet_e_tweet_com_rt = df['retweet_e_tweet_com_rt_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['retweet_e_tweet_com_rt_tratado_media'] = np.array(list(retweet_e_tweet_com_rt))

In [ ]:
x_novo = x

In [ ]:
##Inclui os textos dos twittes (NLTK)
#st = stemmed_count_vect.fit_transform((df['tweet_text']))
#tfidf_transformer = TfidfTransformer()
#x_tfidf = tfidf_transformer.fit_transform(st)
#x_tfidf
#x_novo = x.join(pd.DataFrame(x_tfidf.todense()))

In [ ]:
x_novo.shape

In [ ]:
x_novo.head()

**Com o primeiro conjunto de atributos formado é possível separar o conjunto de dados em treinamento e teste para a elaboração do modelo**

In [ ]:
#Cria um modelo de classificação para o conjunto completo
x_train, x_test, y_train, y_test = train_test_split(x_novo, y, test_size=0.3, random_state=1) 

In [ ]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
np.mean(y_pred == y_test)

In [ ]:
##Seleciona os atributos mais "importantes"
#x_new = SelectKBest(chi2, k=20).fit_transform(x_novo, y)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.3, random_state=1) 

In [ ]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Classificação com RNA
classifier = MLPClassifier(max_iter=1200, random_state=1, activation='tanh', solver='adam') #activation: logistic, relu, tanh, identity | solver: lbfgs, sgd, adam
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))

**Informações de trend topics**

Outra informação que se mostrou de relevância ao longo do trabalho de modelagem foi a relação das postagens de bots com as menções e hashtags listadas nos mais atuais 'trend topics', ou seja, o aparente uso de termos altamente utilizados no momento para possivelmente alavancar a visibilidade da postagem.

Para averiguar essa possibilidade, um sistema de monitoramento dos tópicos mais mencionados foi criado e cada postagem coletada do usuário foi confrontado com os 'trend topics' do período mais próximo. Esse confrontamento gerou um percentual de uso desses tópicos nas postagens dos usuários.

In [ ]:
#Busca os dados de todas as trending topics recuperadas
datafile_trends = "data/sample2/trends_dataclips_qijpjdyxutqsnrteglrjtwjhdjja.csv"
df_trends = pd.read_csv(datafile_trends, header = 0)
#Preenche os valores NaN con 0 apenas para avaliação geral
df_trends = df_trends.fillna(0)
print(len(df_trends))
df_trends.head()

Entre os passos de tratamentos dos dados das "trend topics" está o ajuste dos padrões de data e hora dos registros, tanto dos tópicos monitorados quanto dos próprios tweets.
A seguir são extraídas as datas dos tweets no formato yyyy-mm-dd, dentro da conversão nos próximos trechos foi também necessário ajustar o "timezone" desses dados.

In [ ]:
#Inclui um percentual de trending topics utilizado por tweet
#Para tweet, busca pelos trending topics imediatamente anteriores
df_timeline['Numero de trendings'] = np.array(len(df_timeline))
df_timeline['Numero de trendings'] = 0
df_trends['Trend Date Time Convertido'] = np.array(len(df_trends))

itrend = 0
for x in df_trends['trend_date_time']:
    df_trends['Trend Date Time Convertido'][itrend] = pd.to_datetime(x).strftime("%Y-%m-%d")
    itrend += 1

df_trends.head()   

O relacionamento dos trends e dos tweets foi realizado percorrendo todos os trends armazenados para cada tweet em data anterior ao do tweet e, para cada trend nessa condição, verificou-se no texto do tweet a presença de trendings. Caso esteja presente acumulou-se essa ocorrência, finalizando com a ocorrência de uso de uma trend por cada tweet.
Este trecho demanda de melhorias em desempenho e na inclusão de restrições que reduzam o tempo de ocorrência da trend para mais próximo do tweet.

In [ ]:
itweet = 0
for tweet in df_timeline['tweet_created_at']:
    tweet_date = pd.to_datetime(pd.to_datetime(tweet).strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
    df_temp = df_trends[df_trends['Trend Date Time Convertido'] == tweet_date.strftime("%Y-%m-%d")] 
        
    itrend = 0
    for trend in df_temp['Trend Date Time Convertido']:
        trend_date = pd.to_datetime(pd.to_datetime(trend).strftime("%Y-%m-%d"))
        if trend_date <= tweet_date.tz_convert(None):
            if df_timeline['tweet_text'][itweet].find(df_trends['trend'][itrend]) != -1: 
                df_timeline['Numero de trendings'][itweet] = df_timeline['Numero de trendings'][itweet] + 1
        itrend += 1
    print(itweet)    
    itweet += 1     

Para cada tweet foi armazenados o número de trend topics encontrado.

In [ ]:
df_timeline[df_timeline['Numero de trendings'] > 0].describe()
df_timeline['Numero de trendings'].describe()

In [ ]:
df_timeline

As quantidades de trendings utilizadas em cada tweet foram agrupados por autor (usuário), assim foram incluídos na base de treinamento o número de trendings utilizadas, a média de trendings por tweet desse autor e o número máximo de trendings usado em um mesmo tweet.

In [ ]:
#Reune as informações de trends nos tweets por author
df_result_trend = df_timeline.groupby('tweet_author').agg({'Numero de trendings':lambda col: sum(col)/len(col)}).reset_index()
df_result_trend_max = df_timeline.groupby('tweet_author').agg({'Numero de trendings':lambda col: max(col)}).reset_index()
df_result_trend['trends_media'] = df_result_trend['Numero de trendings']
df_result_trend_max['trends_max'] = df_result_trend_max['Numero de trendings']
df_result_trend_max

In [ ]:
df_handles.head()

In [ ]:
df_trends.head()

In [ ]:
trends_unique = df_trends.trend.unique()

In [ ]:
df_result_merge.head()

Os valores referentes aos trendings do usuário são reunidos ("merged") com os dados gerais do usuário

In [ ]:
df_result_merge = pd.merge(df_result_merge,df_result_trend, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_trend_max, left_on=['handle'], right_on=['tweet_author'])
df_result_merge

In [ ]:
#df_result_merge_trend = df_result_merge
df_result_merge['qtdtrends'] = np.array(list(tam_username))

ttemp = 0
iuser = 0
for user in df_result_merge.tweet_text:
    for trend in trends_unique:
        if user.find(trend) != -1:
            ttemp = ttemp + 1
    print(str(ttemp) + " - " + str(iuser) + " | " + str((iuser/len(df_result_merge.tweet_text))*100) + "%")
    df_result_merge['qtdtrends'][iuser] = ttemp
    iuser = iuser + 1
    ttemp = 0

In [ ]:
df_result_merge.head()

In [ ]:
x_novo_trend = x_novo

Por fim os dados do monitoramento das trendings são incluídos na base de treinamento.

In [ ]:
x_novo_trend['qtdtrends'] = df_result_merge['qtdtrends']
x_novo_trend['trends_media'] = df_result_merge['trends_media']
x_novo_trend['trends_max'] = df_result_merge['trends_max']

In [ ]:
x_novo_trend.head()

**Conjuntos de treinamento e teste**

Os dados reunidos para geração dos modelos são, então, separados em dados de treinamento e teste para a aplicação dos métodos de aprendizagem de máquina - em especial Random Florest, Redes neuronais artificiais e Gradient Boosting.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_novo_trend, y, test_size=0.3, random_state=1) 

In [ ]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
print("Score: " + str(classifier.score(x_test, y_test)))
confusion_matrix(y_test, y_pred)

In [ ]:
classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=1)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
print("Score: " + str(classifier.score(x_test, y_test)))
confusion_matrix(y_test, y_pred)

In [ ]:
importances = classifier.feature_importances_

indices = np.argsort(importances)

fig, ax = plt.subplots(figsize =(10, 6))
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(x_novo_trend.columns)[indices])

**Resultados**

Os resultados ainda demandam de maior avaliação, especialmente com a variação da semente aleatória para os cortes do conjunto de treinamento e para a aplicação dos métodos. Ainda nesse sentido, demanda-se ainda da seleção de modelos baseada na otimização dos hiperparâmetros dos métodos aplicados.

Mesmo com essas demandas, observa-se uma acurácia aproximada de 74% para os métodos (e aproximadamente 70% ao considerar-se o desbalanceamento da base). Valor considerado bom, dado o complexo cenário tratado. 

Importante ponto a ser destacado que o valor da acurácia baseia-se também em um ponto de corte da consistência da classificação, a qual pode variar en 0.0 e 1.0, valores que atrelam-se à probabilidade da classificação, em que por padrão adota-se o corte em 0.5, apesar da aplicação pode gerar um intervalo mais restrito, deslocando a média/mediana das predições. Dito isso e considerando que não deva ser utilizado apenas o corte "bruto" de bot ou não bot, a associação dessa probabilidade permite melhor compreensão do "risco" do usuário ser efetivamente um bot, bem como permite um deslocamento do rigor dessa classificação. 

Os trechos a seguir avaliam a acurácia considerando a mediana das predições como corte, bem como a comparação dos valores preditos nos grupos de usuários previamente (manualmente) classificados como bot ou não, no qual verifica-se uma clara separação dos valores preditos.

In [ ]:
#x_new_trend = SelectKBest(chi2, k=10).fit_transform(x_novo_trend, y)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_new_trend, y, test_size=0.3, random_state=1) 

In [ ]:
#classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
#classifier = classifier.fit(x_train,y_train)
#y_pred = classifier.predict(x_test)
#mean = np.mean(y_pred == y_test)
#balanced = balanced_accuracy_score(y_test, y_pred)
#print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
#confusion_matrix(y_test, y_pred)

In [ ]:
#x_new_trend

In [ ]:
#confusion_matrix(y_test, y_pred)

In [ ]:
y_pred

In [ ]:
classifier.predict_proba(x_test)

In [ ]:
predicted_proba = classifier.predict_proba(x_test)[0]

In [ ]:
y_test

In [ ]:
np.median(classifier.predict_proba(x_test)[:,1])

In [ ]:
threshold = 0.6
predicted = (classifier.predict_proba(x_test)[:,1] >= threshold).astype(bool)

In [ ]:
np.mean(predicted == y_test)

In [ ]:
x_test_geral = x_test
dtf = [x_test, x_train]
x_test_geral = pd.concat(dtf)

In [ ]:
print(len(x_test_geral))
y_test_temp = y_test
y_test_temp.reset_index(drop=True, inplace=True)
y_test_temp[y_test_temp == 1].index
res_geral = classifier.predict_proba(x_test_geral)[y_test_temp.index,1]
res_sim = classifier.predict_proba(x_test_geral)[y_test_temp[y_test_temp == 1].index,1]
res_nao = classifier.predict_proba(x_test_geral)[y_test_temp[y_test_temp == 0].index,1]

np.median(res_sim)
np.median(res_nao)
bplots = plt.boxplot([res_geral, res_nao, res_sim],  vert = 1, patch_artist = False)

In [ ]:
pd.DataFrame({"Não": res_nao}).describe()

In [ ]:
pd.DataFrame({"Sim": res_sim}).describe()

**Comparação com as predições do Botometer**

Visando a avaliar a qualidade da classificação dos modelos gerados, os mesmos usuários passaram pela avaliação da ferramenta Botometer, já bem conhecida e amplamente utilizada (apesar de sua aplicação com enfoque nas publicações em Inglês).

In [ ]:
#Lê os dados da aplicação do botometer
#Busca os dados dos usuários avaliados
datafile_botometer = "data/handles_inct.csv"
df_botometer = pd.read_csv(datafile_botometer, header = 0)
#Preenche os valores NaN con 0 apenas para avaliação geral
df_botometer = df_botometer.fillna(0)
print(len(df_botometer))
df_botometer.head()

In [ ]:
#Avalia os resultados do botometer
a = len(df_botometer['analise_botometer'])
b = len(df_botometer[(df_botometer['É Bot?'] == 'não') | (df_botometer['É Bot?'] == 'Não')]['analise_botometer'])
c = len(df_botometer[(df_botometer['É Bot?'] == 'sim') | (df_botometer['É Bot?'] == 'Sim')]['analise_botometer'])
print(" " + str(a) + " = " + str(b) + " + " + str(c))
botometer_geral = df_botometer['analise_botometer']
botometer_nao   = df_botometer[(df_botometer['É Bot?'] == 'não') | (df_botometer['É Bot?'] == 'Não')]['analise_botometer']
botometer_sim   = df_botometer[(df_botometer['É Bot?'] == 'sim') | (df_botometer['É Bot?'] == 'Sim')]['analise_botometer']

In [ ]:
plt.figure(figsize =(20, 10)) #(11, 6)
bplots = plt.boxplot([botometer_geral/5, botometer_nao/5, botometer_sim/5, res_geral, res_nao, res_sim],  vert = 1, patch_artist = False)
colors = ['blue', 'green', 'red', 'lightblue', 'lightgreen', 'pink']
c = 0
for i, bplot in enumerate(bplots['boxes']):
    bplot.set(color=colors[c], linewidth=3)
    c += 1
    
colorss = ['blue','blue', 'green', 'green', 'red', 'red', 'lightblue', 'lightblue', 'lightgreen', 'lightgreen', 'pink', 'pink' ]    
c3 = 0
for cap in bplots['caps']:
    cap.set(color=colorss[c3], linewidth=3)
    c3 +=1

plt.title("Boxplot da avaliação do Botometer e do novo modelo Pegabot para os dados avaiados no INCT-DD", loc="center", fontsize=18)
plt.xlabel("Agrupados por: (1) Botometer Geral; (2) Botometer apenas considerados não bots; (3) Botometer apenas considerados bots; (4) Novo Pegabot Geral; (5) Novo Pegabot apenas considerados não bots; (6) Novo Pegabot apenas considerados bots")
plt.ylabel("Avaliação do Botometer")

plt.show()

In [ ]:
import scipy
scipy.stats.kruskal(botometer_geral,  botometer_nao,botometer_sim)

In [ ]:
scipy.stats.kruskal(res_geral,  res_nao,res_sim)